# Sub-Workflow Checkpoint

## Overview

This notebook demonstrates **nested workflow checkpointing** - showing how parent and sub-workflows coordinate their execution and state persistence when using `WorkflowExecutor`.

### Workflow Architecture:

```
Parent Workflow:
  DraftWriter → Create initial draft
      ↓
  DraftReviewRouter → Launch sub-workflow
      ↓
  Sub-Workflow (Approval Loop):
    RequestInfoExecutor → Pause for human approval
        ↓ (if revision needed)
    DraftWriter → Improve draft (iteration)
        ↓
    RequestInfoExecutor → Pause again
        ↓ (if approved)
  DraftFinaliser → Publish final result
```

### Key Concepts:

1. **WorkflowExecutor**: Embed complete workflows inside executors
2. **Complex Messages**: Dataclass-based message types for structured data
3. **Nested Checkpointing**: Parent and sub-workflow coordination
4. **Approval Loops**: Iterative revision cycles with human feedback
5. **State Isolation**: Sub-workflow state is separate from parent
6. **Resume Patterns**: Resume parent or sub-workflow independently

### What You Learn:

- Define custom message types with `@dataclass`
- Build sub-workflows with `WorkflowExecutor`
- Handle multi-iteration approval loops
- Resume workflows with pre-supplied human responses
- Manage complex dataclass serialization in checkpoints
- Coordinate parent/sub-workflow execution

## Prerequisites

- Azure OpenAI configured with environment variables
- Azure CLI authentication: `az login`
- Agent Framework installed: `pip install agent-framework`
- Understanding of RequestInfoExecutor from prior notebooks

## Setup and Imports

In [ ]:
from dotenv import load_dotenv
import asyncio
import os
from dataclasses import dataclass
from pathlib import Path
from typing import TYPE_CHECKING, Any

from agent_framework import (
    AgentExecutor,
    AgentExecutorRequest,
    AgentExecutorResponse,
    ChatMessage,
    Executor,
    FileCheckpointStorage,
    RequestInfoExecutor,
    RequestInfoMessage,
    Role,
    WorkflowBuilder,
    WorkflowContext,
    WorkflowExecutor,
    handler,
)
from agent_framework.azure import AzureOpenAIChatClient
from azure.identity import AzureCliCredential

load_dotenv('../../.env')

endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
deployment_name = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")

if TYPE_CHECKING:
    from agent_framework import Workflow
    from agent_framework._workflows._checkpoint import WorkflowCheckpoint

## Configure Checkpoint Storage

In [ ]:
# Define temporary directory for checkpoint files
CHECKPOINT_DIR = Path("./tmp/checkpoints")
CHECKPOINT_DIR.mkdir(parents=True, exist_ok=True)

print(f"Checkpoint directory: {CHECKPOINT_DIR.absolute()}")

## Define Complex Message Types

### Message Hierarchy:

```
DraftTask
    ↓
DraftPackage (includes initial draft)
    ↓
ReviewRequest (RequestInfoMessage subclass)
    ↓ (approval response)
FinalDraft
```

**Why Dataclasses?**
- Type safety for workflow messages
- Automatic serialization for checkpoints
- IDE autocomplete support
- Clear intent in handler signatures

In [ ]:
@dataclass
class DraftTask:
    """Initial task specification for creating a draft."""
    topic: str
    length_words: int = 150


@dataclass
class DraftPackage:
    """Contains a draft along with metadata."""
    topic: str
    draft_text: str
    iteration: int


@dataclass
class ReviewRequest(RequestInfoMessage):
    """Human-in-the-loop approval request with draft content.
    
    Subclasses RequestInfoMessage to enable pause/resume.
    """
    topic: str
    draft_text: str
    iteration: int
    request_type: str = "approval"  # approval or revision


@dataclass
class FinalDraft:
    """Final approved draft ready for publication."""
    topic: str
    content: str
    total_iterations: int


print("✓ Message types defined")

## Create Parent Workflow Executors

### 1. DraftWriter

**Demonstrates:**
- LLM-powered draft generation
- AgentExecutorRequest/Response pattern
- Iteration tracking for revisions

In [ ]:
class DraftWriter(Executor):
    """Generates draft text using an LLM agent."""

    def __init__(self, agent_id: str):
        super().__init__(id="draft_writer")
        self._agent_id = agent_id

    @handler
    async def create_draft(self, task: DraftTask, ctx: WorkflowContext[AgentExecutorRequest]) -> None:
        """Initial draft creation from DraftTask."""
        iteration = 1
        prompt = (
            f"Write a {task.length_words}-word article about {task.topic}. "
            f"Return ONLY the article text."
        )
        print(f"[DraftWriter] Creating initial draft (iter={iteration}) for: {task.topic}")

        # Store iteration in executor state for tracking
        await ctx.set_state({"iteration": iteration, "topic": task.topic})

        await ctx.send_message(
            AgentExecutorRequest(
                messages=[ChatMessage(Role.USER, text=prompt)],
                should_respond=True,
            ),
            target_id=self._agent_id,
        )

    @handler
    async def revise_draft(
        self,
        package: DraftPackage,
        ctx: WorkflowContext[AgentExecutorRequest],
    ) -> None:
        """Revise existing draft based on feedback."""
        iteration = package.iteration + 1
        prompt = (
            f"Improve the following draft article about {package.topic}:\n\n"
            f"{package.draft_text}\n\n"
            f"Make it clearer and more engaging. Return ONLY the improved article text."
        )
        print(f"[DraftWriter] Revising draft (iter={iteration}) for: {package.topic}")

        await ctx.set_state({"iteration": iteration, "topic": package.topic})

        await ctx.send_message(
            AgentExecutorRequest(
                messages=[ChatMessage(Role.USER, text=prompt)],
                should_respond=True,
            ),
            target_id=self._agent_id,
        )

    @handler
    async def package_draft(
        self,
        response: AgentExecutorResponse,
        ctx: WorkflowContext[Any, DraftPackage],
    ) -> None:
        """Package agent response into DraftPackage."""
        state = await ctx.get_state() or {}
        iteration = state.get("iteration", 1)
        topic = state.get("topic", "unknown")
        draft_text = response.agent_run_response.text or ""

        print(f"[DraftWriter] Draft completed (iter={iteration}), {len(draft_text)} chars")

        await ctx.send_message(
            DraftPackage(topic=topic, draft_text=draft_text, iteration=iteration)
        )


print("✓ DraftWriter executor created")

### 2. DraftReviewRouter

**Demonstrates:**
- Sub-workflow launching with WorkflowExecutor
- Routing to embedded approval workflow
- Message transformation for sub-workflow input

In [ ]:
class DraftReviewRouter(Executor):
    """Routes draft to sub-workflow for approval loop."""

    def __init__(self, sub_workflow_executor_id: str):
        super().__init__(id="draft_review_router")
        self._sub_workflow_executor_id = sub_workflow_executor_id

    @handler
    async def route_to_review(
        self,
        package: DraftPackage,
        ctx: WorkflowContext[ReviewRequest],
    ) -> None:
        """Send draft to sub-workflow for human approval."""
        print(f"[DraftReviewRouter] Routing draft to approval sub-workflow (iter={package.iteration})")

        # Transform DraftPackage → ReviewRequest for sub-workflow
        await ctx.send_message(
            ReviewRequest(
                topic=package.topic,
                draft_text=package.draft_text,
                iteration=package.iteration,
                request_type="approval",
            ),
            target_id=self._sub_workflow_executor_id,
        )


print("✓ DraftReviewRouter executor created")

### 3. DraftFinaliser

**Demonstrates:**
- Consuming sub-workflow output
- Final transformation before yield
- Workflow completion

In [ ]:
class DraftFinaliser(Executor):
    """Finalizes approved draft and yields final output."""

    @handler
    async def finalize(
        self,
        package: DraftPackage,
        ctx: WorkflowContext[Any, FinalDraft],
    ) -> None:
        """Package final draft for output."""
        print(f"[DraftFinaliser] Publishing final draft (iter={package.iteration})")

        final = FinalDraft(
            topic=package.topic,
            content=package.draft_text,
            total_iterations=package.iteration,
        )

        await ctx.yield_output(final)


print("✓ DraftFinaliser executor created")

## Build Sub-Workflow (Approval Loop)

### Sub-Workflow Graph:

```
ReviewRequest (start)
    ↓
RequestInfoExecutor → Pause for human approval
    ↓
[Decision based on response]
    ↓ (if "approve")
    Return approved DraftPackage
    ↓ (if "revise")
    Return to parent workflow for revision
```

**Key Pattern:**
- Sub-workflow pauses at RequestInfoExecutor
- Parent workflow is also paused (nested checkpoint)
- Resume supplies human response to sub-workflow
- Sub-workflow returns result to parent

In [ ]:
def create_approval_sub_workflow(
    checkpoint_storage: FileCheckpointStorage,
    parent_writer_id: str,
) -> "Workflow":
    """Build sub-workflow for approval with revision loop."""

    # RequestInfoExecutor for human-in-the-loop pause
    request_info = RequestInfoExecutor(
        id="approval_gate",
        routing={
            "approve": "FINISH",  # Exit sub-workflow, return to parent
            "revise": parent_writer_id,  # Loop back to DraftWriter in parent
        },
    )

    return (
        WorkflowBuilder(max_iterations=5)
        .set_start_executor(request_info)
        .with_checkpointing(checkpoint_storage=checkpoint_storage)
        .build()
    )


print("✓ Sub-workflow factory created")

## Build Parent Workflow

### Parent Graph:

```
DraftWriter (start)
    ↓
DraftAgent (AgentExecutor)
    ↓
DraftWriter (package_draft)
    ↓
DraftReviewRouter
    ↓
ApprovalSubWorkflow (WorkflowExecutor)
    ↓ (if approved)
DraftFinaliser
    ↓ (if revision)
DraftWriter (revise_draft) → loop
```

In [ ]:
def create_parent_workflow(checkpoint_storage: FileCheckpointStorage) -> "Workflow":
    """Build parent workflow with embedded sub-workflow."""

    # Configure LLM agent for draft generation
    endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
    deployment_name = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")
    chat_client = AzureOpenAIChatClient(
        deployment_name=deployment_name,
        endpoint=endpoint,
        credential=AzureCliCredential()
    )
    draft_agent = AgentExecutor(
        chat_client.create_agent(
            instructions="You write clear, engaging article drafts on any topic."
        ),
        id="draft_agent",
    )

    # Parent workflow executors
    draft_writer = DraftWriter(agent_id=draft_agent.id)

    # Create sub-workflow and embed it with WorkflowExecutor
    sub_workflow = create_approval_sub_workflow(
        checkpoint_storage=checkpoint_storage,
        parent_writer_id=draft_writer.id,
    )
    sub_workflow_executor = WorkflowExecutor(sub_workflow, id="approval_sub_workflow")

    draft_router = DraftReviewRouter(sub_workflow_executor_id=sub_workflow_executor.id)
    draft_finaliser = DraftFinaliser()

    # Build parent workflow graph
    return (
        WorkflowBuilder(max_iterations=10)
        # Initial draft creation path
        .add_edge(draft_writer, draft_agent)
        .add_edge(draft_agent, draft_writer)  # package_draft handler
        # Route to sub-workflow for approval
        .add_edge(draft_writer, draft_router)
        .add_edge(draft_router, sub_workflow_executor)
        # Approval path: sub-workflow → finalizer
        .add_edge(sub_workflow_executor, draft_finaliser)
        # Revision path: sub-workflow → writer (loops back)
        .add_edge(sub_workflow_executor, draft_writer)  # revise_draft handler
        .set_start_executor(draft_writer)
        .with_checkpointing(checkpoint_storage=checkpoint_storage)
        .build()
    )


print("✓ Parent workflow factory created")

## Checkpoint Summary Helpers

In [ ]:
def render_checkpoint_summary(checkpoints: list) -> None:
    """Display human-friendly checkpoint metadata."""
    if not checkpoints:
        return

    print("\nCheckpoint summary:")
    for cp in sorted(checkpoints, key=lambda c: c.timestamp):
        summary = RequestInfoExecutor.checkpoint_summary(cp)
        line = f"- {summary.checkpoint_id} | iter={summary.iteration_count}"
        if summary.status:
            line += f" | status={summary.status}"
        if summary.pending_requests:
            for req in summary.pending_requests:
                line += f" | pending={req.executor_id} (req_id={req.request_id})"
        print(line)


def extract_pending_requests(checkpoints: list) -> list:
    """Extract all pending RequestInfo requests from checkpoints."""
    all_pending = []
    for cp in checkpoints:
        pending = RequestInfoExecutor.pending_requests_from_checkpoint(cp)
        all_pending.extend(pending)
    return all_pending


print("✓ Checkpoint helpers created")

## Interactive Approval Session

### Execution Flow:

1. **Initial Run**: Create draft, pause at approval gate
2. **Inspect Checkpoint**: View pending approval requests
3. **User Decision**: Approve or request revision
4. **Resume**: Supply response to checkpoint
5. **Iteration**: If revision, loop back to step 1
6. **Completion**: If approved, finalize and exit

In [ ]:
async def run_interactive_approval():
    """Run workflow with interactive approval loop."""

    # Clear existing checkpoints
    for file in CHECKPOINT_DIR.glob("*.json"):
        file.unlink()

    checkpoint_storage = FileCheckpointStorage(storage_path=CHECKPOINT_DIR)
    workflow = create_parent_workflow(checkpoint_storage=checkpoint_storage)

    print("\n" + "="*70)
    print("Interactive Draft Approval Demo")
    print("="*70)

    # Start initial workflow run
    task = DraftTask(topic="Python async programming", length_words=150)
    print(f"\nStarting draft workflow for: {task.topic}\n")

    checkpoint_id: str | None = None

    try:
        async for event in workflow.run_stream(message=task):
            print(f"Event: {event}")
    except Exception as e:
        # Workflow pauses at RequestInfoExecutor - expected behavior
        if "RequestInfo" in str(e):
            print(f"\n⏸️  Workflow paused for approval: {e}")
        else:
            raise

    # Approval loop
    max_iterations = 3
    for iteration_num in range(max_iterations):
        print("\n" + "="*70)
        print(f"Approval Iteration {iteration_num + 1}")
        print("="*70)

        # List checkpoints and find pending requests
        all_checkpoints = await checkpoint_storage.list_checkpoints()
        if not all_checkpoints:
            print("No checkpoints found. Workflow may have completed.")
            break

        workflow_id = all_checkpoints[0].workflow_id
        render_checkpoint_summary(all_checkpoints)

        pending = extract_pending_requests(all_checkpoints)
        if not pending:
            print("\nNo pending approval requests. Workflow completed!")
            break

        # Display pending request details
        print(f"\nFound {len(pending)} pending approval request(s):")
        for idx, req in enumerate(pending):
            # Access ReviewRequest fields from pending message
            review_req = req.request_message
            print(f"\n[{idx}] Request ID: {req.request_id}")
            print(f"    Executor: {req.executor_id}")
            print(f"    Topic: {review_req.topic}")
            print(f"    Iteration: {review_req.iteration}")
            print(f"    Draft Preview: {review_req.draft_text[:200]}...")

        # Get user decision
        user_choice = input(
            "\nDo you approve this draft? (approve/revise/quit): "
        ).strip().lower()

        if user_choice == "quit":
            print("Exiting approval loop.")
            break

        if user_choice not in ["approve", "revise"]:
            print(f"Invalid choice '{user_choice}'. Please choose 'approve' or 'revise'.")
            continue

        # Find checkpoint with pending request
        target_checkpoint = None
        for cp in all_checkpoints:
            if RequestInfoExecutor.pending_requests_from_checkpoint(cp):
                target_checkpoint = cp
                break

        if not target_checkpoint:
            print("No checkpoint with pending request found.")
            break

        checkpoint_id = target_checkpoint.checkpoint_id

        # Resume workflow with user response
        print(f"\nResuming from checkpoint: {checkpoint_id}")
        print(f"Supplying response: {user_choice}\n")

        new_workflow = create_parent_workflow(checkpoint_storage=checkpoint_storage)
        responses = {pending[0].request_id: user_choice}

        try:
            async for event in new_workflow.run_stream_from_checkpoint(
                checkpoint_id,
                checkpoint_storage=checkpoint_storage,
                responses=responses,
            ):
                print(f"Resumed Event: {event}")

            # If approved, workflow completes
            if user_choice == "approve":
                print("\n✅ Draft approved and finalized!")
                break
        except Exception as e:
            if "RequestInfo" in str(e):
                print(f"\n⏸️  Workflow paused again for next approval: {e}")
            else:
                raise

    print("\n" + "="*70)
    print("Interactive approval session completed")
    print("="*70)

## Run the Interactive Demo

In [ ]:
await run_interactive_approval()

## Expected Output Pattern

```
======================================================================
Interactive Draft Approval Demo
======================================================================

Starting draft workflow for: Python async programming

[DraftWriter] Creating initial draft (iter=1) for: Python async programming
Event: ExecutorInvokeEvent(executor_id=draft_writer)
Event: ExecutorInvokeEvent(executor_id=draft_agent)
[DraftWriter] Draft completed (iter=1), 450 chars
Event: ExecutorInvokeEvent(executor_id=draft_writer)
[DraftReviewRouter] Routing draft to approval sub-workflow (iter=1)
Event: ExecutorInvokeEvent(executor_id=draft_review_router)
Event: ExecutorInvokeEvent(executor_id=approval_sub_workflow)

⏸️  Workflow paused for approval: RequestInfo pending at approval_gate

======================================================================
Approval Iteration 1
======================================================================

Checkpoint summary:
- abc123... | iter=3 | pending=approval_gate (req_id=xyz789...)

Found 1 pending approval request(s):

[0] Request ID: xyz789...
    Executor: approval_gate
    Topic: Python async programming
    Iteration: 1
    Draft Preview: Python async programming enables concurrent execution using asyncio...

Do you approve this draft? (approve/revise/quit): revise

Resuming from checkpoint: abc123...
Supplying response: revise

[DraftWriter] Revising draft (iter=2) for: Python async programming
Resumed Event: ExecutorInvokeEvent(executor_id=draft_writer)
Resumed Event: ExecutorInvokeEvent(executor_id=draft_agent)
[DraftWriter] Draft completed (iter=2), 480 chars

⏸️  Workflow paused again for next approval: RequestInfo pending at approval_gate

======================================================================
Approval Iteration 2
======================================================================

Checkpoint summary:
- def456... | iter=6 | pending=approval_gate (req_id=uvw321...)

Found 1 pending approval request(s):

[0] Request ID: uvw321...
    Executor: approval_gate
    Topic: Python async programming
    Iteration: 2
    Draft Preview: Python async programming revolutionizes concurrent execution...

Do you approve this draft? (approve/revise/quit): approve

Resuming from checkpoint: def456...
Supplying response: approve

[DraftFinaliser] Publishing final draft (iter=2)
Resumed Event: ExecutorInvokeEvent(executor_id=draft_finaliser)
Resumed Event: WorkflowOutputEvent(output=FinalDraft(...))

✅ Draft approved and finalized!

======================================================================
Interactive approval session completed
======================================================================
```

## Key Takeaways

### 1. Complex Message Types

```python
@dataclass
class ReviewRequest(RequestInfoMessage):
    """Subclass RequestInfoMessage for checkpoint serialization."""
    topic: str
    draft_text: str
    iteration: int
    request_type: str = "approval"
```

**Benefits:**
- Type-safe message passing
- Automatic checkpoint serialization
- IDE autocomplete support
- Clear handler signatures

**Requirements:**
- Use `@dataclass` decorator
- Subclass `RequestInfoMessage` for HITL messages
- All fields must be JSON-serializable

### 2. WorkflowExecutor Pattern

```python
# Create sub-workflow
sub_workflow = WorkflowBuilder().set_start_executor(...).build()

# Embed in parent as executor
sub_executor = WorkflowExecutor(sub_workflow, id="approval_sub_workflow")

# Use in parent graph
builder.add_edge(router, sub_executor)
builder.add_edge(sub_executor, finalizer)  # approved path
builder.add_edge(sub_executor, writer)     # revision path
```

**Use Cases:**
- Reusable approval workflows
- Multi-agent coordination
- Modular workflow composition
- Recursive task decomposition

### 3. Nested Checkpointing

**Checkpoint Hierarchy:**
```
Parent Checkpoint (iter=3)
    ↓
    executor_states:
        draft_writer: {iteration: 1, topic: "..."}
        approval_sub_workflow: {...}
    ↓
    Sub-Workflow Checkpoint (embedded)
        ↓
        executor_states:
            approval_gate: {pending_request_id: "xyz789..."}
```

**Resume Behavior:**
- Resume parent → resumes sub-workflow automatically
- Sub-workflow state restored from parent checkpoint
- Responses routed to sub-workflow RequestInfoExecutor

### 4. Approval Loop Patterns

#### Route-Based Iteration
```python
RequestInfoExecutor(
    routing={
        "approve": "FINISH",        # Exit sub-workflow
        "revise": parent_writer_id, # Loop back to parent executor
    }
)
```

**Flow:**
1. Sub-workflow pauses at RequestInfoExecutor
2. User supplies "revise" response
3. Message sent to `parent_writer_id` in **parent** workflow
4. Parent workflow re-executes DraftWriter.revise_draft()
5. New draft routed back to sub-workflow
6. Repeat until "approve"

### 5. Handler Overloading

```python
class DraftWriter(Executor):
    @handler
    async def create_draft(self, task: DraftTask, ctx) -> None:
        # Initial creation
        ...

    @handler
    async def revise_draft(self, package: DraftPackage, ctx) -> None:
        # Revision iteration
        ...

    @handler
    async def package_draft(self, response: AgentExecutorResponse, ctx) -> None:
        # Transform agent response
        ...
```

**Routing Logic:**
- Framework selects handler based on **message type**
- `DraftTask` → `create_draft()`
- `DraftPackage` → `revise_draft()`
- `AgentExecutorResponse` → `package_draft()`

### 6. Pending Request Inspection

```python
# Get all checkpoints
checkpoints = await storage.list_checkpoints()

# Extract pending requests
pending = []
for cp in checkpoints:
    pending.extend(RequestInfoExecutor.pending_requests_from_checkpoint(cp))

# Access request details
for req in pending:
    print(f"Request ID: {req.request_id}")
    print(f"Executor: {req.executor_id}")
    review_msg = req.request_message  # ReviewRequest instance
    print(f"Topic: {review_msg.topic}")
    print(f"Draft: {review_msg.draft_text}")
```

### 7. Resume with Responses

```python
# Build response map: request_id → user_response
responses = {pending[0].request_id: "approve"}

# Resume from checkpoint
async for event in workflow.run_stream_from_checkpoint(
    checkpoint_id,
    checkpoint_storage=storage,
    responses=responses,  # Pre-supplied human responses
):
    process(event)
```

**Multi-Request Pattern:**
```python
responses = {
    "req_id_1": "approve",
    "req_id_2": "revise",
    "req_id_3": "approve",
}
```

### 8. Production Considerations

#### State Isolation
- Sub-workflow state is **independent** from parent
- Use `shared_state` for cross-workflow communication
- Avoid tight coupling between parent and sub-workflows

#### Error Handling
```python
try:
    async for event in workflow.run_stream(...):
        process(event)
except Exception as e:
    if "RequestInfo" in str(e):
        # Expected pause - save checkpoint_id
        checkpoint_id = extract_checkpoint_from_error(e)
    else:
        # Unexpected error - log and alert
        raise
```

#### Checkpoint Cleanup
```python
# Delete completed workflow checkpoints
for cp in completed_checkpoints:
    await storage.delete_checkpoint(cp.checkpoint_id)

# Archive old checkpoints
await storage.archive_checkpoints(older_than=30_days)
```

#### Message Type Registry
```python
# Ensure all custom message types are importable
from typing import get_type_hints

# Framework uses type hints for handler routing
handler_signature = get_type_hints(executor.handler_method)
message_type = handler_signature['message_param']
```